'common_keywords_3_plus'를 단어사전기반으로 단어 통합
-> 해당 총평 title 있는 행만 추려냄
-> 'Processed_Review'열에서 okt사용하여 기본형으로 바꾸기
-> 비슷한 단어 그룹에서 중복 제거(ex. 크다, 넉넉하다 -> 크다)
-> 정사이즈/크다/작다에 각각 속하는 단어들 추려내기

In [23]:
import os
import pandas as pd
from konlpy.tag import Okt

okt = Okt()

# 사이즈 관련 키워드 설정
size_keywords = [
    "사이즈", "정사이즈", "치수", "크다", "작다", "적당하다", "맞다", "끼다", "타이트", "여유롭다", 
    "보통", "엠", "에스", "라지", "xs", "s", "m", "l", "xl", "ws", "허리", "골반", "어깨", 
    "엉덩이", "히프", "스판끼", "XS", "S", "M", "L", "스몰", "널찍하다", "편하다", "여유", 
    "오버사이즈", "오버", "오버핏", "넉넉하다", "조이다", "핏하다", "업", "다운", "미디움", "미듐", "미디", "크게", "작게"
]

# 비슷한 단어 그룹 설정
similar_words = {
    '정사이즈': ['정사이즈', '맞다', '적당하다', '보통'],
    '크다': ['크다', '여유롭다', '여유', '널찍하다', '넉넉하다', '오버사이즈', '오버', '박시', '오버핏'],
    '작다': ['작다', '끼다', '타이트', '조이다', '핏하다', '붙다']
}

# 리뷰의 기본형 변환 함수
def get_lemmas(text, okt):
    return [word for word, pos in okt.pos(text, norm=True, stem=True)]

# 그룹 내 중복된 단어 제거 함수
def remove_duplicate_group_words(lemmas, similar_words):
    unique_lemmas = lemmas[:]
    
    # 비슷한 단어 그룹에서 중복 제거
    for group, words in similar_words.items():
        group_words_in_review = [lemma for lemma in lemmas if lemma in words]
        if len(group_words_in_review) > 1:
            for word in group_words_in_review[1:]:
                if word in unique_lemmas:
                    unique_lemmas.remove(word)  # 첫 번째 단어만 남기고 나머지를 제거
    
    return unique_lemmas

# 각 그룹별 비율 계산 함수
def calculate_group_ratios(reviews, similar_words, size_keywords, df):
    # 사이즈 관련 키워드가 포함된 리뷰 수 계산 (분모)
    size_filtered_reviews = df['common_keywords_3_plus'].apply(lambda x: any(word in x for word in size_keywords))
    size_related_review_count = size_filtered_reviews.sum()

    # 분모 값이 0일 경우 1로 설정하여 나누기 오류 방지
    total_reviews = size_related_review_count if size_related_review_count > 0 else 1
    group_ratios = {}
    
    # 비슷한 단어 그룹 비율 계산 (사이즈 관련 키워드가 포함된 리뷰에 대해서만)
    for group, words in similar_words.items():
        count = 0
        for review in reviews[size_filtered_reviews]:
            if any(word in review for word in words):
                count += 1
        group_ratios[group] = count / total_reviews
    
    return group_ratios

# 하나의 파일을 처리하는 함수
def process_file(file_path):
    df = pd.read_csv(file_path)

    # '정 사이즈' -> '정사이즈'
    df['Processed_Review'] = df['Processed_Review'].apply(
        lambda x: x.replace('정 사이즈', '정사이즈') if pd.notnull(x) else x)

    # '온 버핏' -> '오버핏'
    df['Processed_Review'] = df['Processed_Review'].apply(
        lambda x: x.replace('온 버핏', '오버핏') if pd.notnull(x) else x)

    # '버핏' -> '오버핏'
    df['common_keywords_3_plus'] = df['common_keywords_3_plus'].apply(
        lambda x: ', '.join(['오버핏' if keyword.strip() == '버핏' else keyword for keyword in str(x).split(',')]) if pd.notnull(x) else x)

    # 'common_keywords_3_plus' 열을 리스트로 변환
    df['common_keywords_3_plus'] = df['common_keywords_3_plus'].apply(
        lambda x: [item.strip() for item in x.split(',')] if isinstance(x, str) else [])

    # 'Processed_Review' 열에 형태소 분석 적용
    df['lemmatized'] = df['Processed_Review'].apply(lambda x: get_lemmas(str(x), okt))

    # 리뷰에서 비슷한 단어 그룹의 단어 중 중복 제거
    df['lemmatized_unique'] = df['lemmatized'].apply(lambda x: remove_duplicate_group_words(x, similar_words))

    # 각 그룹별 비율 계산
    lemmatized_reviews_unique = df['lemmatized_unique']
    group_ratios = calculate_group_ratios(lemmatized_reviews_unique, similar_words, size_keywords, df)

    # 파일명 추출
    filename = os.path.basename(file_path)
    
    # 소수점 둘째 자리까지 반올림하여 출력
    print(f"{filename}")
    for group, ratio in group_ratios.items():
        print(f'{group}: {round(ratio*100, 2)}%')
    print()  # 파일마다 구분을 위해 빈 줄 출력

# 디렉터리 내 모든 파일 처리 함수
def process_all_csv(directory_path):
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory_path, filename)
            process_file(file_path)

# 'merged_hashtag' 디렉터리 내 모든 파일 처리
directory_path = './merged_hashtag'
process_all_csv(directory_path)

1645762_keywords.csv
정사이즈: 45.32%
크다: 34.8%
작다: 13.16%

1875663_keywords.csv
정사이즈: 36.01%
크다: 18.01%
작다: 9.32%

1875664_keywords.csv
정사이즈: 37.72%
크다: 22.49%
작다: 14.53%

1944554_keywords.csv
정사이즈: 51.78%
크다: 37.26%
작다: 17.53%

2551401_keywords.csv
정사이즈: 40.94%
크다: 24.32%
작다: 14.64%

2978106_keywords.csv
정사이즈: 19.06%
크다: 43.75%
작다: 7.81%

3295891_keywords.csv
정사이즈: 38.46%
크다: 34.62%
작다: 7.69%

3977452_keywords.csv
정사이즈: 37.74%
크다: 16.98%
작다: 45.28%

3977488_keywords.csv
정사이즈: 29.41%
크다: 5.88%
작다: 44.12%

4018731_keywords.csv
정사이즈: 17.05%
크다: 19.32%
작다: 20.45%

460G 컷 헤비 피그먼트 티셔츠-차콜-_keywords.csv
정사이즈: 33.08%
크다: 43.26%
작다: 3.82%

ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이_keywords.csv
정사이즈: 28.62%
크다: 42.15%
작다: 5.23%

Deep One Tuck Sweat Shorts [Grey]_keywords.csv
정사이즈: 21.14%
크다: 28.73%
작다: 5.69%

DOODLE HEART HALF T WHITE GREYISH BLUE_keywords.csv
정사이즈: 31.91%
크다: 40.08%
작다: 3.89%

TAG OG TEE - WHITE_keywords.csv
정사이즈: 28.33%
크다: 44.83%
작다: 5.42%

[16수 코마사] 2PACK SMALL ARCH T-SHIRT WHITE _ BLACK_ke

In [61]:
import pandas as pd

file_path = './merged_hashtag/[2PACK] EL 스트리트 아트워크 오버핏 반팔티 2종 2PACK_keywords.csv'
df = pd.read_csv(file_path)

# '버핏' -> '오버핏'
df['common_keywords_3_plus'] = df['common_keywords_3_plus'].apply(lambda x: ', '.join(['오버핏' if keyword.strip() == '버핏' else keyword for keyword in str(x).split(',')]) if pd.notnull(x) else x)

In [62]:
from konlpy.tag import Okt
okt = Okt()

# 비슷한 단어 그룹 설정
similar_words = {
    '정사이즈': ['정사이즈', '맞다', '적당하다', '보통'],
    '크다': ['크다', '여유롭다', '여유', '널찍하다', '넉넉하다', '오버사이즈', '오버', '박시', '오버핏'],
    '작다': ['작다', '끼다', '타이트', '조이다', '핏하다', '붙다']
}

# 'keyword' 열에 '사이즈'가 포함된 행 필터링
size_related_reviews = df[df['common_keywords_3_plus'].str.contains('사이즈', na=False)]

# 리뷰의 기본형 변환 함수
def get_lemmas(text, okt):
    return [word for word, pos in okt.pos(text, norm=True, stem=True)]

# 각 리뷰의 형태소를 기본형으로 변환하여 저장
size_related_reviews['lemmatized'] = size_related_reviews['Processed_Review'].apply(lambda x: get_lemmas(str(x), okt))

# 각 그룹별 비율 계산 함수
def calculate_group_ratios(reviews, similar_words):
    total_reviews = len(reviews)
    group_ratios = {}
    
    for group, words in similar_words.items():
        count = 0
        for review in reviews:
            # 리뷰 내에서 해당 그룹의 단어가 존재하는지 확인
            if any(word in review for word in words):
                count += 1
        group_ratios[group] = count / total_reviews if total_reviews > 0 else 0
    
    return group_ratios

# 'lemmatized' 열에서 비율 계산
lemmatized_reviews = size_related_reviews['lemmatized']
group_ratios = calculate_group_ratios(lemmatized_reviews, similar_words)

# 소수점 둘째 자리까지 반올림하여 출력
for group, ratio in group_ratios.items():
    print(f'{group}: {round(ratio*100, 2)}%')

정사이즈: 40.83%
크다: 30.77%
작다: 10.06%


C:\Users\USER\AppData\Local\Temp\ipykernel_10136\494092199.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size_related_reviews['lemmatized'] = size_related_reviews['Processed_Review'].apply(lambda x: get_lemmas(str(x), okt))


In [88]:
size_related_reviews

,Review,Processed_Review,common_keywords_all,common_keywords_3_plus
0,사이즈 잘 맞구요 스타일도 좋아서 자주 입는 옷이에요.,사이즈 잘 맞고요 스타일도 좋아서 자주 입는 옷이에요,NaN,"맞다, 사이즈, 스타일"
1,예쁘고 무낭하네요 사이즈 한치수 크게해도 될듯해요,예쁘고 무나 하네요 사이즈 한 치수 크게 해도 될듯해요,사이즈,"무, 사이즈, 치수, 크게, 해도"
2,"제 체형에서 2는 확실히 크고, 1은 딱 붙습니다. 흰옷은 요즘 입기엔 적당한 두께...",제 체형에서는 확실히 크고 은 딱 붙습니다 흰옷은 요즘 입기엔 적당한 두께이나 비침...,NaN,"두께, 붙다, 비침, 사이즈, 얼룩, 의사, 이유, 재다, 체형, ..."
7,사이즈 적당하고 예쁘고 가볍게 입기좋은거 같습니다,사이즈 적당하고 예쁘고 가볍게 입기 좋은 거 같습니다,사이즈,"가볍다, 사이즈, 적당하다"
9,"사이즈랑 핏은 생각한 느낌 그대로라 이쁜데,,,, 저는 검은색 검은색 선택했는데 검...",사이즈랑 핏은 생각한 느낌 그대로라 이쁜데 저는 검은색 검은색 선택했는데 검은색 흰...,흰색,"검은색, 귀찮다, 그대로, 사이즈, 선택, 오다, 핏, 흰색"
...,...,...,...,...
889,너무 이쁘네요 사이즈도 딱 좋은 오버핏입니다 하,너무 이쁘네요 사이즈도 딱 좋은 오버핏입니다 하,NaN,"사이즈, 오버핏"
892,사이즈 너무 적당히 좋아요 옷도 예뻐서 많이 입고 다니는 중이에요 역시 무신사 무신...,사이즈 너무 적당히 좋아요 옷도 예뻐서 많이 입고 다니는 중이에요 역시 무신사 무신...,NaN,"사이즈, 적당하다, 중이, 파다"
901,사이즈도 널널하고 편하고 좋습니다 배송도 빨라요,사이즈도 널찍하고 편하고 좋습니다 배송도 빨라요,NaN,"널찍하다, 배송, 빠르다, 사이즈, 편하다"
916,디자인도 심플하고 무난하게 입기 좋습니다! 저는 사이즈 오버핏으로 샀어용,디자인도 심플하고 무난하게 입기 좋습니다 저는 사이즈 온 버핏으로 샀어요,NaN,"디자인, 무난, 오버핏, 사이즈, 심플"


In [ ]:
import pandas as pd
import os
from konlpy.tag import Okt

okt = Okt()

folder_path = './merged_hashtag'

# 비슷한 단어 그룹 설정
similar_words = {
    '정사이즈': ['정사이즈', '맞다', '적당하다', '보통'],
    '크다': ['크다', '여유롭다', '여유', '널찍하다', '넉넉하다', '오버사이즈', '오버'],
    '작다': ['작다', '끼다', '타이트', '조이다', '핏하다']
}

# 리뷰의 기본형 변환 함수
def get_lemmas(text, okt):
    return [word for word, pos in okt.pos(text, norm=True, stem=True)]

# 각 그룹별 비율 계산 함수
def calculate_group_ratios(reviews, similar_words):
    total_reviews = len(reviews)
    group_ratios = {}
    
    for group, words in similar_words.items():
        count = 0
        for review in reviews:
            # 리뷰 내에서 해당 그룹의 단어가 존재하는지 확인
            if any(word in review for word in words):
                count += 1
        group_ratios[group] = count / total_reviews if total_reviews > 0 else 0
    
    return group_ratios

# 'merged_hashtag' 폴더 내의 모든 CSV 파일에 대해 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # CSV 파일 읽기
        df = pd.read_csv(file_path)
        
        # '버핏' -> '오버핏' 변환
        df['common_keywords_3_plus'] = df['common_keywords_3_plus'].apply(
            lambda x: ', '.join(['오버핏' if keyword.strip() == '버핏' else keyword for keyword in str(x).split(',')]) 
            if pd.notnull(x) else x
        )
        
        # 'common_keywords_3_plus' 열에 '사이즈'가 포함된 행 필터링
        size_related_reviews = df[df['common_keywords_3_plus'].str.contains('사이즈', na=False)].copy()
        
        # 형태소 분석 후 'lemmatized' 열에 저장
        size_related_reviews['lemmatized'] = size_related_reviews['Processed_Review'].apply(lambda x: get_lemmas(str(x), okt))
        
        # 'lemmatized' 열에서 비율 계산
        lemmatized_reviews = size_related_reviews['lemmatized']
        group_ratios = calculate_group_ratios(lemmatized_reviews, similar_words)
        
        # 결과 출력
        print(f"\nFile: {filename}")
        for group, ratio in group_ratios.items():
            print(f'{group}: {round(ratio * 100, 2)}%')

In [48]:
# '크게'에 대해 Okt 형태소 분석을 적용한 결과 확인
from konlpy.tag import Okt

# Initialize Okt tokenizer
okt = Okt()

# Apply the tokenizer to the word '크게'
result = okt.pos('한 치수 크게 해도 될듯', norm=True, stem=True)
# result = okt.pos('크지도 작지도 않다')
result

[('하다', 'Verb'),
 ('치수', 'Noun'),
 ('크게', 'Noun'),
 ('해도', 'Noun'),
 ('되다', 'Verb')]

In [44]:
# 비율(전체 행)
import pandas as pd
from konlpy.tag import Okt
okt = Okt()

file_path = './merged_hashtag/[2PACK] EL 스트리트 아트워크 오버핏 반팔티 2종 2PACK_keywords.csv'
df = pd.read_csv(file_path)

# '정 사이즈' -> '정사이즈'
df['Processed_Review'] = df['Processed_Review'].apply(
    lambda x: x.replace('정 사이즈', '정사이즈') if pd.notnull(x) else x)

# '온 버핏' -> '오버핏' 
df['Processed_Review'] = df['Processed_Review'].apply(
    lambda x: x.replace('온 버핏', '오버핏') if pd.notnull(x) else x)

# 비슷한 단어 그룹 설정
similar_words = {
    '정사이즈': ['정사이즈', '맞다', '적당하다', '보통'],
    '크다': ['크다', '여유롭다', '여유', '널찍하다', '넉넉하다', '오버사이즈', '오버', '박시', '오버핏'],
    '작다': ['작다', '끼다', '타이트', '조이다', '핏하다', '붙다']
}

# 리뷰의 기본형 변환 함수
def get_lemmas(text, okt):
    return [word for word, pos in okt.pos(text, norm=True, stem=True)]

# 'Processed_Review' 열에 형태소 분석 적용
df['lemmatized'] = df['Processed_Review'].apply(lambda x: get_lemmas(str(x), okt))

# 각 그룹별 비율 계산 함수
def calculate_group_ratios(reviews, similar_words):
    total_reviews = len(reviews)
    group_ratios = {}
    
    for group, words in similar_words.items():
        count = 0
        for review in reviews:
            # 리뷰 내에서 해당 그룹의 단어가 존재하는지 확인
            if any(word in review for word in words):
                count += 1
        group_ratios[group] = count / total_reviews if total_reviews > 0 else 0
    
    return group_ratios

# 'lemmatized' 열에서 비율 계산
lemmatized_reviews = df['lemmatized']
group_ratios = calculate_group_ratios(lemmatized_reviews, similar_words)

# 소수점 둘째 자리까지 반올림하여 출력
for group, ratio in group_ratios.items():
    print(f'{group}: {round(ratio*100, 2)}%')

정사이즈: 15.62%
크다: 9.7%
작다: 3.77%


In [47]:
# 비율(관련 행)
import pandas as pd
from konlpy.tag import Okt
okt = Okt()

file_path = './merged_hashtag/[2PACK] EL 스트리트 아트워크 오버핏 반팔티 2종 2PACK_keywords.csv'
df = pd.read_csv(file_path)

# '정 사이즈' -> '정사이즈'
df['Processed_Review'] = df['Processed_Review'].apply(
    lambda x: x.replace('정 사이즈', '정사이즈') if pd.notnull(x) else x)

# '온 버핏' -> '오버핏' 
df['Processed_Review'] = df['Processed_Review'].apply(
    lambda x: x.replace('온 버핏', '오버핏') if pd.notnull(x) else x)

# 비슷한 단어 그룹 설정
similar_words = {
    '정사이즈': ['정사이즈', '맞다', '적당하다', '보통'],
    '크다': ['크다', '여유롭다', '여유', '널찍하다', '넉넉하다', '오버사이즈', '오버', '박시', '오버핏'],
    '작다': ['작다', '끼다', '타이트', '조이다', '핏하다', '붙다']
}

# 리뷰의 기본형 변환 함수
def get_lemmas(text, okt):
    return [word for word, pos in okt.pos(text, norm=True, stem=True)]
    
# 'Processed_Review' 열에 형태소 분석 적용
df['lemmatized'] = df['Processed_Review'].apply(lambda x: get_lemmas(str(x), okt))
# 'lemmatized' 열에서 사이즈 관련 단어가 포함된 행 필터링
size_related_reviews = df[df['lemmatized'].apply(lambda x: any(word in x for words in similar_words.values() for word in words))]

# 각 그룹별 비율 계산 함수
def calculate_group_ratios(reviews, similar_words):
    total_reviews = len(reviews)
    group_ratios = {}
    
    for group, words in similar_words.items():
        count = 0
        for review in reviews:
            # 리뷰 내에서 해당 그룹의 단어가 존재하는지 확인
            if any(word in review for word in words):
                count += 1
        group_ratios[group] = count / total_reviews if total_reviews > 0 else 0
    
    return group_ratios

# 'lemmatized' 열에서 비율 계산
lemmatized_reviews_filtered = size_related_reviews['lemmatized']
group_ratios_filtered = calculate_group_ratios(lemmatized_reviews_filtered, similar_words)

# 소수점 둘째 자리까지 반올림하여 출력
for group, ratio in group_ratios_filtered.items():
    print(f'{group}: {round(ratio*100, 2)}%')

정사이즈: 62.23%
크다: 38.63%
작다: 15.02%


In [43]:
# 행 개수(관련 행)
import pandas as pd
from konlpy.tag import Okt
okt = Okt()

file_path = './merged_hashtag/[2PACK] EL 스트리트 아트워크 오버핏 반팔티 2종 2PACK_keywords.csv'
df = pd.read_csv(file_path)

# '정 사이즈' -> '정사이즈'
df['Processed_Review'] = df['Processed_Review'].apply(
    lambda x: x.replace('정 사이즈', '정사이즈') if pd.notnull(x) else x)

# '온 버핏' -> '오버핏' 
df['Processed_Review'] = df['Processed_Review'].apply(
    lambda x: x.replace('온 버핏', '오버핏') if pd.notnull(x) else x)

# 비슷한 단어 그룹 설정
similar_words = {
    '정사이즈': ['정사이즈', '맞다', '적당하다', '보통'],
    '크다': ['크다', '여유롭다', '여유', '널찍하다', '넉넉하다', '오버사이즈', '오버', '박시', '오버핏'],
    '작다': ['작다', '끼다', '타이트', '조이다', '핏하다', '붙다']
}

# 리뷰의 기본형 변환 함수
def get_lemmas(text, okt):
    return [word for word, pos in okt.pos(text, norm=True, stem=True)]

# 'Processed_Review' 열에 형태소 분석 적용하여 'lemmatized' 열 생성
df['lemmatized'] = df['Processed_Review'].apply(lambda x: get_lemmas(str(x), okt))

# 'lemmatized' 열에서 사이즈 관련 단어가 포함된 행 필터링
size_related_reviews = df[df['lemmatized'].apply(lambda x: any(word in x for words in similar_words.values() for word in words))]

# 각 그룹별 행의 개수 계산 함수
def calculate_group_counts(reviews, similar_words):
    group_counts = {}
    
    for group, words in similar_words.items():
        count = 0
        for review in reviews:
            # 리뷰 내에서 해당 그룹의 단어가 존재하는지 확인
            if any(word in review for word in words):
                count += 1
        group_counts[group] = count
    
    return group_counts

# 'lemmatized' 열에서 각 그룹의 행 개수 계산
lemmatized_reviews_filtered = size_related_reviews['lemmatized']
group_counts_filtered = calculate_group_counts(lemmatized_reviews_filtered, similar_words)

# 결과 출력
group_counts_filtered

{'정사이즈': 145, '크다': 90, '작다': 35}

In [50]:
import pandas as pd
from konlpy.tag import Okt
import os

# 형태소 분석 및 비율 계산 함수
def process_csv(file_path):
    okt = Okt()
    
    # CSV 파일 읽기
    df = pd.read_csv(file_path)

    # '정 사이즈' -> '정사이즈'
    df['Processed_Review'] = df['Processed_Review'].apply(
        lambda x: x.replace('정 사이즈', '정사이즈') if pd.notnull(x) else x)

    # '온 버핏' -> '오버핏'
    df['Processed_Review'] = df['Processed_Review'].apply(
        lambda x: x.replace('온 버핏', '오버핏') if pd.notnull(x) else x)

    # 비슷한 단어 그룹 설정
    similar_words = {
        '정사이즈': ['정사이즈', '맞다', '적당하다', '보통'],
        '크다': ['크다', '여유롭다', '여유', '널찍하다', '넉넉하다', '오버사이즈', '오버', '박시', '오버핏'],
        '작다': ['작다', '끼다', '타이트', '조이다', '핏하다', '붙다']
    }

    # 리뷰의 기본형 변환 함수
    def get_lemmas(text, okt):
        return [word for word, pos in okt.pos(text, norm=True, stem=True)]

    # 'Processed_Review' 열에 형태소 분석 적용
    df['lemmatized'] = df['Processed_Review'].apply(lambda x: get_lemmas(str(x), okt))

    # 'lemmatized' 열에서 사이즈 관련 단어가 포함된 행 필터링
    size_related_reviews = df[df['lemmatized'].apply(lambda x: any(word in x for words in similar_words.values() for word in words))]

    # 각 그룹별 비율 계산 함수
    def calculate_group_ratios(reviews, similar_words):
        total_reviews = len(reviews)
        group_ratios = {}
        
        for group, words in similar_words.items():
            count = 0
            for review in reviews:
                # 리뷰 내에서 해당 그룹의 단어가 존재하는지 확인
                if any(word in review for word in words):
                    count += 1
            group_ratios[group] = count / total_reviews if total_reviews > 0 else 0
        
        return group_ratios

    # 'lemmatized' 열에서 비율 계산
    lemmatized_reviews_filtered = size_related_reviews['lemmatized']
    group_ratios_filtered = calculate_group_ratios(lemmatized_reviews_filtered, similar_words)

    # 소수점 둘째 자리까지 반올림하여 결과 반환
    return {group: round(ratio * 100, 2) for group, ratio in group_ratios_filtered.items()}

# 폴더 내의 모든 CSV 파일에 대해 함수 적용
def process_all_csv(folder_path):
    results = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            result = process_csv(file_path)
            results[filename] = result
    return results

# 'merged_hashtag' 폴더 내 모든 CSV 파일에 적용
folder_path = './merged_hashtag'
all_results = process_all_csv(folder_path)

# 결과 출력
for filename, ratios in all_results.items():
    print(f"파일명: {filename}")
    for group, ratio in ratios.items():
        print(f'{group}: {ratio}%')
    print()

파일명: 1645762_keywords.csv
정사이즈: 61.78%
크다: 46.72%
작다: 18.15%

파일명: 1875663_keywords.csv
정사이즈: 63.05%
크다: 32.02%
작다: 22.66%

파일명: 1875664_keywords.csv
정사이즈: 61.41%
크다: 38.04%
작다: 27.17%

파일명: 1944554_keywords.csv
정사이즈: 62.39%
크다: 44.78%
작다: 21.79%

파일명: 2551401_keywords.csv
정사이즈: 62.37%
크다: 37.98%
작다: 23.34%

파일명: 2978106_keywords.csv
정사이즈: 32.63%
크다: 74.74%
작다: 13.68%

파일명: 3295891_keywords.csv
정사이즈: 60.61%
크다: 51.52%
작다: 12.12%

파일명: 3977452_keywords.csv
정사이즈: 46.81%
크다: 21.28%
작다: 57.45%

파일명: 3977488_keywords.csv
정사이즈: 38.46%
크다: 11.54%
작다: 61.54%

파일명: 4018731_keywords.csv
정사이즈: 34.62%
크다: 34.62%
작다: 44.23%

파일명: 460G 컷 헤비 피그먼트 티셔츠-차콜-_keywords.csv
정사이즈: 56.38%
크다: 50.21%
작다: 7.0%

파일명: ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이_keywords.csv
정사이즈: 50.48%
크다: 53.37%
작다: 9.62%

파일명: Deep One Tuck Sweat Shorts [Grey]_keywords.csv
정사이즈: 46.15%
크다: 58.46%
작다: 11.28%

파일명: DOODLE HEART HALF T WHITE GREYISH BLUE_keywords.csv
정사이즈: 54.19%
크다: 50.32%
작다: 6.45%

파일명: TAG OG TEE - WHITE_keywords.csv
정사이즈: 